In [3]:
import sys
sys.path.append('')
import pandas as pd
import numpy as np
import xgboost as xgb

from gini import *

ModuleNotFoundError: No module named 'gini'

In [ ]:
train = pd.read_csv('data/train.csv', na_values=-1)
train = train.fillna(train.median())

In [2]:
train = train.drop(train.columns[train.columns.str.startswith('ps_calc') == True], axis=1)
train = train.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                    'ps_ind_18_bin','ps_car_03_cat', 'ps_car_10_cat'], axis=1)

NameError: name 'train' is not defined

In [6]:
train = pd.get_dummies(train, columns=train.columns[train.columns.str.endswith('cat')==True])

In [8]:
test = pd.read_csv('data/test.csv', na_values=-1)
test = test.fillna(test.median())
test = test.drop(test.columns[test.columns.str.startswith('ps_calc') == True], axis=1)
test = test.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                    'ps_ind_18_bin','ps_car_03_cat', 'ps_car_10_cat'], axis=1)
test = pd.get_dummies(test, columns=test.columns[test.columns.str.endswith('cat')==True])
X_test = test.drop('id', axis=1)
xg_test = xgb.DMatrix(X_test.values)

In [9]:
X = train.drop(['target','id'], axis=1)
y = train.target
X = X.values
y = y.values

In [11]:
param = {'tree_method': 'gpu_hist', 
         'silent': True, 
         'eval_metric': 'auc',
         'objective': 'binary:logistic',
         'eta': 0.01, 
         'max_depth': 5, 
         'gamma': 0.3, 
         'min_child_weight': 7, 
         'subsample': 0.7, 
         'colsample_bytree': 0.6, 
         'max_delta_step': 5
        }

In [10]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=False, random_state=123)

In [28]:
from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold

score = []
pred = pd.DataFrame({'id': test.id,'target': np.zeros_like(test.id)})

for i in random_list:
    model_num = 1
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
    for train_index, val_index in skf.split(X, y):
        print("TRAIN:", train_index, "TEST:", val_index)
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]
        # Set Var for XGB
        xg_train = xgb.DMatrix(X_train, label=y_train)
        xg_val = xgb.DMatrix(X_val, label=y_val)
        watchlist = [(xg_train, 'train'), (xg_val, 'test')]
        #
        xgb_skf = xgb.train(param, xg_train, 10000, watchlist, early_stopping_rounds=100
                            , feval=gini_xgb, maximize=True, verbose_eval=100)
        print(xgb_skf.best_score)
        score.append(xgb_skf.best_score)
        joblib.dump(xgb_skf, 'model/xgb'+str(model_num)+'.pkl')
        model_num += 1

    model_list = glob.glob('model/*.pkl')

    for model in model_list:
        xgb_pred = joblib.load(model)
        pred_lap = xgb_pred.predict(xg_test)
        pred['target'] += pred_lap 

pred['target'] = pred['target']/(len(random_list)*5)

TRAIN: [     1      2      3 ..., 595207 595208 595210] TEST: [     0      6     13 ..., 595203 595209 595211]
[0]	train-auc:0.596931	test-auc:0.593371	train-gini:0.193575	test-gini:0.18689
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.632009	test-auc:0.624597	train-gini:0.264018	test-gini:0.249193
[200]	train-auc:0.633934	test-auc:0.626282	train-gini:0.267867	test-gini:0.252563
[300]	train-auc:0.639606	test-auc:0.629994	train-gini:0.279211	test-gini:0.259988
[400]	train-auc:0.646119	test-auc:0.633663	train-gini:0.292238	test-gini:0.267326
[500]	train-auc:0.651996	test-auc:0.637115	train-gini:0.303992	test-gini:0.27423
[600]	train-auc:0.65727	test-auc:0.639682	train-gini:0.31454	test-gini:0.279364
[700]	train-auc:0.661516	test-auc:0.64102	train-gini:0.323032	test-gini:0.282041
[800]	train-auc:0.664908	test-auc:0.64194	train-gini:0.329816	test-gini:0.28388
[900]	train-auc:

[700]	train-auc:0.661551	test-auc:0.63851	train-gini:0.323102	test-gini:0.277019
[800]	train-auc:0.665138	test-auc:0.639887	train-gini:0.330276	test-gini:0.279774
[900]	train-auc:0.668459	test-auc:0.640803	train-gini:0.336918	test-gini:0.281607
[1000]	train-auc:0.671255	test-auc:0.641503	train-gini:0.34251	test-gini:0.283006
[1100]	train-auc:0.673714	test-auc:0.641824	train-gini:0.347427	test-gini:0.283648
[1200]	train-auc:0.676157	test-auc:0.642334	train-gini:0.352314	test-gini:0.284669
[1300]	train-auc:0.678369	test-auc:0.64262	train-gini:0.356738	test-gini:0.285239
[1400]	train-auc:0.68049	test-auc:0.642769	train-gini:0.36098	test-gini:0.285538
[1500]	train-auc:0.68243	test-auc:0.642898	train-gini:0.364861	test-gini:0.285796
[1600]	train-auc:0.684332	test-auc:0.643011	train-gini:0.368665	test-gini:0.286022
[1700]	train-auc:0.686336	test-auc:0.643295	train-gini:0.372672	test-gini:0.28659
[1800]	train-auc:0.688206	test-auc:0.643442	train-gini:0.376413	test-gini:0.286884
[1900]	train-a

Stopping. Best iteration:
[1483]	train-auc:0.681131	test-auc:0.648808	train-gini:0.362262	test-gini:0.297616

0.297616
TRAIN: [     1      2      4 ..., 595208 595210 595211] TEST: [     0      3     11 ..., 595193 595202 595209]
[0]	train-auc:0.597991	test-auc:0.597886	train-gini:0.195786	test-gini:0.196254
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.63186	test-auc:0.624954	train-gini:0.26372	test-gini:0.249908
[200]	train-auc:0.634428	test-auc:0.626926	train-gini:0.268857	test-gini:0.253851
[300]	train-auc:0.639293	test-auc:0.629827	train-gini:0.278586	test-gini:0.259654
[400]	train-auc:0.64592	test-auc:0.634085	train-gini:0.291841	test-gini:0.26817
[500]	train-auc:0.651645	test-auc:0.636942	train-gini:0.303291	test-gini:0.273884
[600]	train-auc:0.656462	test-auc:0.639251	train-gini:0.312924	test-gini:0.278502
[700]	train-auc:0.660519	test-auc:0.641017	train-gini:0.32

[1600]	train-auc:0.684278	test-auc:0.644431	train-gini:0.368557	test-gini:0.288862
[1700]	train-auc:0.686152	test-auc:0.644543	train-gini:0.372304	test-gini:0.289087
[1800]	train-auc:0.68802	test-auc:0.644601	train-gini:0.376039	test-gini:0.289201
[1900]	train-auc:0.689683	test-auc:0.64464	train-gini:0.379365	test-gini:0.28928
[2000]	train-auc:0.691594	test-auc:0.644715	train-gini:0.383187	test-gini:0.289429
Stopping. Best iteration:
[1957]	train-auc:0.690732	test-auc:0.644733	train-gini:0.381464	test-gini:0.289465

0.289465
TRAIN: [     0      1      2 ..., 595207 595209 595210] TEST: [     4      5      8 ..., 595206 595208 595211]
[0]	train-auc:0.596066	test-auc:0.595435	train-gini:0.19131	test-gini:0.193144
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.63262	test-auc:0.626825	train-gini:0.26524	test-gini:0.253651
[200]	train-auc:0.634336	test-auc:0.62743	train-gini:0.

Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.632887	test-auc:0.62541	train-gini:0.265774	test-gini:0.250821
[200]	train-auc:0.63454	test-auc:0.626288	train-gini:0.26908	test-gini:0.252576
[300]	train-auc:0.640029	test-auc:0.629488	train-gini:0.280058	test-gini:0.258977
[400]	train-auc:0.646562	test-auc:0.633054	train-gini:0.293124	test-gini:0.266107
[500]	train-auc:0.652536	test-auc:0.635523	train-gini:0.305072	test-gini:0.271046
[600]	train-auc:0.65739	test-auc:0.637686	train-gini:0.314781	test-gini:0.275372
[700]	train-auc:0.66171	test-auc:0.639413	train-gini:0.323421	test-gini:0.278827
[800]	train-auc:0.665179	test-auc:0.640152	train-gini:0.330358	test-gini:0.280304
[900]	train-auc:0.668369	test-auc:0.640854	train-gini:0.336738	test-gini:0.281709
[1000]	train-auc:0.671313	test-auc:0.641402	train-gini:0.342626	test-gini:0.282805
[1100]	train-auc:0.673995	test-auc:0.641

...

0.258432
TRAIN: [     0      1      2 ..., 595207 595210 595211] TEST: [    13     15     33 ..., 595193 595208 595209]
[0]	train-auc:0.598167	test-auc:0.59325	train-gini:0.195826	test-gini:0.187396
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.633914	test-auc:0.621161	train-gini:0.267828	test-gini:0.242321
[200]	train-auc:0.636324	test-auc:0.622706	train-gini:0.272649	test-gini:0.245412
[300]	train-auc:0.641161	test-auc:0.625954	train-gini:0.282322	test-gini:0.251909
[400]	train-auc:0.647017	test-auc:0.629612	train-gini:0.294034	test-gini:0.259225
[500]	train-auc:0.652987	test-auc:0.6329	train-gini:0.305974	test-gini:0.265801
[600]	train-auc:0.65821	test-auc:0.635356	train-gini:0.316419	test-gini:0.270711
[700]	train-auc:0.662455	test-auc:0.636833	train-gini:0.324911	test-gini:0.273667
[800]	train-auc:0.666234	test-auc:0.638183	train-gini:0.332468	test-gini:0.276366
[900

[1400]	train-auc:0.679833	test-auc:0.646534	train-gini:0.359667	test-gini:0.293069
[1500]	train-auc:0.681957	test-auc:0.646778	train-gini:0.363913	test-gini:0.293556
[1600]	train-auc:0.683772	test-auc:0.646828	train-gini:0.367545	test-gini:0.293655
Stopping. Best iteration:
[1577]	train-auc:0.683367	test-auc:0.64687	train-gini:0.366735	test-gini:0.293739

0.293739
TRAIN: [     0      1      2 ..., 595209 595210 595211] TEST: [     8     12     13 ..., 595198 595202 595208]
[0]	train-auc:0.596008	test-auc:0.598936	train-gini:0.19209	test-gini:0.19689
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.632502	test-auc:0.630191	train-gini:0.265005	test-gini:0.260382
[200]	train-auc:0.634722	test-auc:0.631421	train-gini:0.269444	test-gini:0.262841
[300]	train-auc:0.640011	test-auc:0.634678	train-gini:0.280023	test-gini:0.269357
[400]	train-auc:0.645763	test-auc:0.637655	train-gini:

[500]	train-auc:0.651811	test-auc:0.639564	train-gini:0.303623	test-gini:0.279129
[600]	train-auc:0.656915	test-auc:0.642218	train-gini:0.31383	test-gini:0.284435
[700]	train-auc:0.660982	test-auc:0.643887	train-gini:0.321964	test-gini:0.287774
[800]	train-auc:0.664935	test-auc:0.645176	train-gini:0.329871	test-gini:0.290352
[900]	train-auc:0.668035	test-auc:0.645821	train-gini:0.33607	test-gini:0.291642
[1000]	train-auc:0.67083	test-auc:0.646263	train-gini:0.341661	test-gini:0.292527
[1100]	train-auc:0.673442	test-auc:0.646597	train-gini:0.346883	test-gini:0.293193
[1200]	train-auc:0.675553	test-auc:0.646836	train-gini:0.351106	test-gini:0.293671
[1300]	train-auc:0.677773	test-auc:0.64704	train-gini:0.355546	test-gini:0.294079
[1400]	train-auc:0.679844	test-auc:0.647211	train-gini:0.359689	test-gini:0.294422
[1500]	train-auc:0.681862	test-auc:0.647373	train-gini:0.363724	test-gini:0.294746
[1600]	train-auc:0.683899	test-auc:0.647509	train-gini:0.367798	test-gini:0.295017
[1700]	train-

[1300]	train-auc:0.67819	test-auc:0.646168	train-gini:0.35638	test-gini:0.292337
[1400]	train-auc:0.680498	test-auc:0.64641	train-gini:0.360995	test-gini:0.292821
[1500]	train-auc:0.682698	test-auc:0.646741	train-gini:0.365397	test-gini:0.293482
[1600]	train-auc:0.684716	test-auc:0.646765	train-gini:0.369433	test-gini:0.293531
[1700]	train-auc:0.686737	test-auc:0.646877	train-gini:0.373474	test-gini:0.293755
[1800]	train-auc:0.688637	test-auc:0.646818	train-gini:0.377274	test-gini:0.293635
Stopping. Best iteration:
[1719]	train-auc:0.68705	test-auc:0.646888	train-gini:0.3741	test-gini:0.293776

0.293776
TRAIN: [     1      2      4 ..., 595209 595210 595211] TEST: [     0      3      7 ..., 595167 595184 595206]
[0]	train-auc:0.595811	test-auc:0.59641	train-gini:0.191013	test-gini:0.194209
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.632704	test-auc:0.62612	train-gini:0.

[2000]	train-auc:0.692673	test-auc:0.641926	train-gini:0.385346	test-gini:0.283853
Stopping. Best iteration:
[1928]	train-auc:0.691292	test-auc:0.641991	train-gini:0.382583	test-gini:0.283982

0.283982
TRAIN: [     0      1      2 ..., 595209 595210 595211] TEST: [    11     13     16 ..., 595200 595206 595208]
[0]	train-auc:0.596014	test-auc:0.595672	train-gini:0.192603	test-gini:0.188397
Multiple eval metrics have been passed: 'test-gini' will be used for early stopping.

Will train until test-gini hasn't improved in 100 rounds.
[100]	train-auc:0.633365	test-auc:0.626283	train-gini:0.266729	test-gini:0.252565
[200]	train-auc:0.635261	test-auc:0.627575	train-gini:0.270522	test-gini:0.255149
[300]	train-auc:0.639881	test-auc:0.630371	train-gini:0.279761	test-gini:0.260742
[400]	train-auc:0.646357	test-auc:0.634432	train-gini:0.292714	test-gini:0.268864
[500]	train-auc:0.652425	test-auc:0.637699	train-gini:0.30485	test-gini:0.275397
[600]	train-auc:0.657427	test-auc:0.640205	train-gini:

In [24]:
len(random_list)*5

15

In [27]:
pred.to_csv('sub/sub25.csv', index=False)

In [30]:
sub25 = pd.read_csv('sub/sub25.csv')

In [35]:
pred['target'] = pred['target']-(sub25['target']/50)

In [36]:
pred['target'] = (pred['target']+sub25['target'])/2

In [38]:
pred.to_csv('sub/sub26.csv', index=False)